<a href="https://colab.research.google.com/github/codingcat101/StealthTask/blob/main/SoccerPLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Re-Identification (Re-ID) in a single video feed

In [2]:
# Install the ultralytics package from PyPI
!pip install ultralytics
#install supervision for tracking
!pip install supervision

In [3]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.151 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.9/112.6 GB disk)


In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [18]:
#Import All the Required Libraries
from ultralytics import YOLO

#Load the YOLO Model
model = YOLO("/content/drive/MyDrive/best.pt")

#Object Detection
results = model.predict(source = "/content/drive/MyDrive/soccer.mp4", save=True)

#Tracking
results = model.track(source = "/content/drive/MyDrive/soccer.mp4", save=True, persist=True)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) /content/drive/MyDrive/soccer.mp4: 384x640 1 ball, 16 players, 2 referees, 52.7ms
video 1/1 (frame 2/375) /content/drive/MyDrive/soccer.mp4: 384x640 18 players, 2 referees, 50.7ms
video 1/1 (frame 3/375) /content/drive/MyDrive/soccer.mp4: 384x640 1 ball, 16 players, 2 referees, 39.3ms
video 1/1 (frame 4/375) /content/drive/MyDrive/soccer.mp4: 384x640 1 ball, 14 players, 2 referees, 39.1ms
video 1/1 (frame 5/375) /content/drive/M

Basic Functions

In [40]:
# utils.py

import cv2

def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def save_video(output_video_frames, output_video_path):
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 24,
                          (output_video_frames[0].shape[1], output_video_frames[0].shape[0]))
    for frame in output_video_frames:
        out.write(frame)
    out.release()

def get_center_of_bbox(bbox):
    x1, y1, x2, y2 = bbox
    return int((x1 + x2) / 2), int((y1 + y2) / 2)

def get_bbox_width(bbox):
    return bbox[2] - bbox[0]


Core Player Re-ID Tracking

In [44]:
# tracker.py

import cv2
import os
import pickle
import numpy as np
from ultralytics import YOLO
import supervision as sv


class Tracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.tracker = sv.ByteTrack

    def detect_frames(self, frames):
        batch_size = 20
        detections = []
        for i in range(0, len(frames), batch_size):
            batch = self.model.predict(frames[i:i + batch_size], conf=0.2)
            detections.extend(batch)
        return detections

    def get_player_tracks(self, frames, read_from_stub=False, stub_path=None):
        if read_from_stub and stub_path and os.path.exists(stub_path):
            with open(stub_path, 'rb') as f:
                return pickle.load(f)

        detections = self.detect_frames(frames)
        tracks = []

        for frame_idx, detection in enumerate(detections):
            cls_names = detection.names
            cls_inv = {v: k for k, v in cls_names.items()}
            det_supervision = sv.Detections.from_ultralytics(detection)

            # Convert goalkeepers to players
            for idx, class_id in enumerate(det_supervision.class_id):
                if cls_names[class_id] == "goalkeeper":
                    det_supervision.class_id[idx] = cls_inv["player"]

            tracked_detections = self.tracker.update_with_detections(det_supervision)

            frame_tracks = {}
            for obj in tracked_detections:
                bbox = obj[0].tolist()
                cls_id = obj[3]
                track_id = obj[4]

                if cls_names[cls_id] == "player":
                    frame_tracks[track_id] = {"bbox": bbox}
            tracks.append(frame_tracks)

        if stub_path:
            with open(stub_path, 'wb') as f:
                pickle.dump(tracks, f)

        return tracks

    def draw_ellipse(self, frame, bbox, color, track_id=None):
        y2 = int(bbox[3])
        x_center, _ = get_center_of_bbox(bbox)
        width = get_bbox_width(bbox)

        cv2.ellipse(frame,
                    center=(x_center, y2),
                    axes=(int(width), int(0.35 * width)),
                    angle=0.0,
                    startAngle=-45,
                    endAngle=235,
                    color=color,
                    thickness=2)

        if track_id is not None:
            cv2.rectangle(frame, (x_center - 20, y2 + 15), (x_center + 20, y2 + 35), color, cv2.FILLED)
            cv2.putText(frame, str(track_id), (x_center - 10, y2 + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

        return frame

    def annotate_frames(self, video_frames, tracks):
        output_frames = []
        for i, frame in enumerate(video_frames):
            frame = frame.copy()
            for track_id, player in tracks[i].items():
                frame = self.draw_ellipse(frame, player["bbox"], (0, 0, 255), track_id)
            output_frames.append(frame)
        return output_frames


 Full Re-ID Pipeline

In [42]:
# main.py


import os

def main():
    video_path = "/content/drive/MyDrive/soccer.mp4"
    model_path = "/content/drive/MyDrive/best.pt"
    output_path = "output_videos/output_15s.avi"
    stub_path = "/content/drive/MyDrive/player_detection.pkl"

    print("Reading video...")
    frames = read_video(video_path)

    print("Initializing Tracker...")
    tracker = Tracker(model_path)

    print("Detecting and tracking players...")
    tracks = tracker.get_player_tracks(frames, read_from_stub=False, stub_path=stub_path)

    print("Drawing annotations...")
    output_frames = tracker.annotate_frames(frames, tracks)

    print("Saving video...")
    os.makedirs("output_videos", exist_ok=True)
    save_video(output_frames, output_path)

    print("Done! Output saved to:", output_path)

if __name__ == "__main__":
    main()


Reading video...
Initializing Tracker...
Detecting and tracking players...

0: 384x640 1 ball, 16 players, 2 referees, 42.2ms
1: 384x640 1 ball, 18 players, 2 referees, 42.2ms
2: 384x640 1 ball, 16 players, 2 referees, 42.2ms
3: 384x640 1 ball, 14 players, 2 referees, 42.2ms
4: 384x640 1 ball, 14 players, 2 referees, 42.2ms
5: 384x640 1 ball, 16 players, 2 referees, 42.2ms
6: 384x640 1 ball, 15 players, 2 referees, 42.2ms
7: 384x640 15 players, 1 referee, 42.2ms
8: 384x640 1 ball, 16 players, 2 referees, 42.2ms
9: 384x640 1 ball, 15 players, 2 referees, 42.2ms
10: 384x640 1 ball, 16 players, 2 referees, 42.2ms
11: 384x640 1 ball, 15 players, 2 referees, 42.2ms
12: 384x640 1 ball, 13 players, 2 referees, 42.2ms
13: 384x640 15 players, 2 referees, 42.2ms
14: 384x640 14 players, 2 referees, 42.2ms
15: 384x640 20 players, 2 referees, 42.2ms
16: 384x640 14 players, 2 referees, 42.2ms
17: 384x640 15 players, 2 referees, 42.2ms
18: 384x640 15 players, 2 referees, 42.2ms
19: 384x640 15 players

In [43]:
from google.colab import files
files.download('/content/output_videos/output.avi')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>